In [ ]:
from random import * 
from copy import deepcopy

In [ ]:
class UndirectedGraph:
    def __init__(self):
        self.edges = {}
        self.pheromones = {}
        
    def add_pheromones(self, x,  y, ph):
        self.pheromones[(x, y)] += ph
        self.pheromones[(y, x)] += ph
        
    def add_edge(self, x, y):
        if not x in self.edges.keys():
            self.edges[x] = [y]
        else:
            self.edges[x].append(y)
        
        if not y in self.edges.keys():
            self.edges[y] = [x]
        else:
            self.edges[y].append(x)
            
        self.pheromones[(x, y)] = 0
        self.pheromones[(y, x)] = 0
    
    def nr_of_vertices():
        return len(self.edges.keys())
    
    def get_pheromones(self, x, y):
        return self.pheromones[(x, y)]; 
    
    def get_neighbours(self, x):
        return self.edges[x]
    
    def decrease_pheromones_on_all_edges(self, coef):
        for edge in self.pheromones.keys():
            self.pheromones[edge] -= coef

    def __str__(self):
        return str(self.pheromones)

In [ ]:
class Ant:
    def __init__(self):
        self.solution = []
    
    def fitness(self, problem):
        # defined as the difference between the numbere of correct words and guessed words

        sol = self.solution + problem.get_column_words(self.solution)
        
        return len([word for word in problem.words if word not in sol])
    
    def update(self, x):
        self.solution.append(x)
        
    def __str__(self):
        return str(self.solution)

In [ ]:
class Problem:
    def __init__(self, file_name):
        self.load_data(file_name)

    def load_data(self, file_name):
        with open(file_name, 'r') as f:
            self.words = [line.strip(" ").strip("\n") for line in f]
            self.n = len(self.words)
            self.words = self.words + self.get_column_words(self.words)
            self.graph = UndirectedGraph()
            for i in range(len(self.words) - 1):
                for j in range(i + 1, len(self.words)):
                    self.graph.add_edge(self.words[i], self.words[j])
            
    def get_column_words(self, words):
        return [ "".join([ words[row][col]  for row in range(len(words))])for col in range(len(words[0]))]

In [ ]:
class Controller:
    def __init__(self, data_file_path, params_file_path):
        self.problem = Problem(data_file_path)
        self.no_runs = 0 
        self.no_genes = 0
        self.local_pheromone_increase = 0
        self.global_pheromone_increase = 0
        self.evaporation_coefficient = 0
        self.probability_for_best = 0
        self.load_parameters(params_file_path)
        self.population = []
        self.no_steps = len(self.problem.words) // 2 - 1
        self.graph = deepcopy(self.problem.graph)
        
    def iteration(self):
        for word in self.problem.words:
            ant = Ant()
            ant.update(word)
            self.population.append(ant)
            
        for step in range(self.no_steps):
            for i, ant in enumerate(self.population):
                current_word = ant.solution[step]
                neighbours = self.get_neighbours(ant, current_word)
                
#                 print("ant {}".format(i))
#                 print("solution: {}".format(ant))
#                 print("neighbours: {}".format(neighbours))
                
                if (random() < self.probability_for_best):
                    next_word = self.get_best_neighbour(neighbours, current_word)
#                     print("I am choosing the best")
                else:
                    next_word = choice(self.graph.get_neighbours(current_word))
#                     print("I am choosing randomly")
#                 print("my choice is {}".format(next_word))
                ant.update(next_word)

    
        best_ant = self.get_best_ant()

        if best_ant != None:
            self.spread_pheromones(best_ant)
        
    def spread_pheromones(self, ant):
        x = ant.solution[0]
        for y in ant.solution[1:]:
            self.graph.add_pheromones(x, y, self.local_pheromone_increase)
            x = y 
            
    def get_neighbours(self, ant, current_word):
        neighbours = deepcopy(self.graph.get_neighbours(current_word))
        neighbours = [ neighbour for neighbour in neighbours if neighbour not in ant.solution]
        if len(neighbours) == 0:
            return deepcopy(self.graph.get_neighbours(current_word)) 
        
        return neighbours

    def get_best_ant(self):
        population = sorted(self.population, key = lambda ant: ant.fitness(self.problem))
        if (population[0] != population[1]):
            return population[0]
        return None
    
    def get_best_neighbour(self, neighbours, current_word):
        next_word = neighbours.pop()
        for neighbour in neighbours: 
            if self.graph.get_pheromones(current_word, next_word) < self.graph.get_pheromones(current_word, neighbour):
                next_word = neighbour
            return next_word
        
    def run(self):
#         print(self.problem.words)
        for i in range(self.no_gens):
            self.iteration()
      
            best_ant = self.get_best_ant()
        
            if best_ant.fitness(self.problem) == 0: 
                return best_ant
            
#             PRINT EVERY GENERATION
#             for index, ant in enumerate(self.population):
#                 print("ant {}".format(index))
#                 print("solution {}".format(ant.solution))
#                 print("fitness {}".format(ant.fitness(self.problem)))
#             print('-' * 100)

            self.evaporation()
            self.population = [] 
        self.graph = deepcopy(self.problem.graph)
        return best_ant
    
    def evaporation(self):
        self.graph.decrease_pheromones_on_all_edges(self.evaporation_coefficient)

    
    def spread_pheromones_on_path(self, best_ant):
        ant = deepcopy(best_ant)
        x1 = ant.soution.pop()
        while len(ant.solution) > 0:
            x2 = ant.solution.pop()
            self.graph.add_pheromones(x1, x2, self.global_pheromone_increase)
            x1 = x2
    
    def load_parameters(self, params_file_name):
        with open(params_file_name, 'r') as file:
            
            line = file.readline().strip()
            self.no_runs = int(line.replace("number of runs:",""))
            
            line = file.readline().strip()
            self.no_gens = int(line.replace("number of generations:",""))
            
            line = file.readline().strip()
            self.local_pheromone_increase = float(line.replace("local pheromone increase:",""))
            
            line = file.readline().strip()
            self.global_pheromone_increase = float(line.replace("global pheromone increase:",""))
            
            line = file.readline().strip()
            self.evaporation_coefficient = float(line.replace("evaporation coefficient:",""))
            
            line = file.readline().strip()
            self.probability_for_best = float(line.replace("probability for best:",""))
    

In [ ]:
class UI:
    def __init__(self, controller):
        self.controller = controller
        
    def main(self):
        print("computing one solution..")
        best_ant = self.controller.run()
        print("{} fitness {}".format(best_ant.solution, best_ant.fitness(self.controller.problem)))

controller = Controller("medium.txt", "params.txt")
ui = UI(controller)
ui.main()